In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch


In [ ]:

# default: Load the model on the available device(s)
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="bfloat16", device_map="auto"
# )

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
)

# default processer
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
min_pixels = 256 * 28 * 28
max_pixels = 420 * 28 * 28
processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels
)


In [ ]:

resized_height = 420
resized_width = 640
# Messages containing multiple images and a text query
messages = [
    {
        "role": "user",
        "content": [
            # {
            #     "type": "image",
            #     "image": "data/images/t1.png",
            #     "resized_height": resized_height,
            #     "resized_width": resized_width,
            # },
            # {
            #     "type": "image",
            #     "image": "data/images/t2.png",
            #     "resized_height": resized_height,
            #     "resized_width": resized_width,
            # },
            # {
            #     "type": "image",
            #     "image": "data/images/t3.png",
            #     "resized_height": resized_height,
            #     "resized_width": resized_width,
            # },
            # {
            #     "type": "text",
            #     "text": "This was set 1 of 3 images. ",
            # },
            {
                "type": "image",
                "image": "data/images/t1.png",
                "resized_height": resized_height,
                "resized_width": resized_width,
            },
            {
                "type": "image",
                "image": "data/images/t2.png",
                "resized_height": resized_height,
                "resized_width": resized_width,
            },
            {
                "type": "image",
                "image": "data/images/t3.png",
                "resized_height": resized_height,
                "resized_width": resized_width,
            },
            {
                "type": "text",
                "text": "You are given frames from a video of a person hitting a cricket shot. Comment on their stance, was it good or bad",
            },
        ],
    }
]


In [ ]:

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference
generated_ids = model.generate(**inputs, max_new_tokens=256)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)
